In [ ]:
### importing required packages

In [457]:
import pandas as pd
import requests
import concurrent.futures
import numpy as np
import sys
from os.path import exists
import re
import itertools
import orjson
import time
from datetime import date
import urllib.parse
import os
import xml.etree.ElementTree
from bs4 import BeautifulSoup
from os.path import exists
import json
import csv
import time

In [458]:
CLEANR = re.compile('<.*?>')
def cleanhtml(raw_html):
    cleantext =re.sub(CLEANR, '', str(raw_html))
    return cleantext

In [459]:
out_csv = 'ExpressCarWash.csv'
input_folder = "ExpressCarWash"
baseurl = "https://www.exppresscarwash.com/LocateOurOutlets"

csv_exists = exists(out_csv)
if csv_exists:
    print(f"csv exists in path {out_csv},removing now ....")
    os.remove(out_csv)
df = pd.DataFrame(columns=['Company_Name','Pincode','State','City','Company_Url','Logo_Url','Rating,Reviews','Contact_Person_1','Contact_Person_2','Email','Website','Address','Head_Office','Listed_In,About_Us','Opening_Closing_Time','Weekly_Services','Mobile1','Mobile2','Mobile3','Mobile4','Mobile5','Mobile6','tollfree1','tollfree2','tollfree3','stdcode','Phone1','Phone2','Phone3','Phone4','Phone5','fax_cut','Fax_prefix','Fax_Number1','Fax_Number2','Tag,Image_Urls','Established_Year','Tagline','Feature-Tags','Tagline2','Lat','Lon'])
header_list=df.columns
print(f"creating new CSV...")
df.to_csv(out_csv,sep="|",index=False,header=False)

creating new CSV...


In [460]:
parent_dir = "/home/justdial/data/input"
path = os.path.join(parent_dir, input_folder) 
file_exists = exists(path)
if not file_exists :
        os.makedirs(path)
else:
    print("Path Already Exits")

Path Already Exits


# fetch_details() function

In [461]:
def fetch_details(spage,main_cat):
    spage = re.sub('<!--(.*?)-->','',str(spage))
    spage = re.sub('&times;','x',str(spage))
    spage = re.sub('amp;','',str(spage))
    spage = re.sub('&#8211;','-',str(spage))
    spage = re.sub('&#8217;','',str(spage))
    spage = re.sub('&#038;','&',str(spage))
    spage = re.sub('&#8243;','Inches',str(spage))
    #spage = re.sub('&quot;','Inches',str(spage))
    spage = re.sub('&nbsp;','',str(spage))
    
    cut1 = re.findall('<div class="col-md-6 bg-yellow">(.*?)<div class="col-md-2">',str(spage))[0]
    # company name
    try:
        company_name = re.findall('<span>(.*?)<',str(cut1))[0]
        company_name = re.sub('</li>\s*</ul>','',str(company_name))
        company_name = re.sub('</li>',' ~ ',str(company_name))
        #company_name = " ".join(company_name)
        company_name = cleanhtml(company_name)
    
    except:
        company_name = ""
    
    # city
    try:
        city = re.findall('class="address-text">(.*?)</span>',str(cut1))[0]
        city = re.sub('</li>\s*</ul>','',str(city))
        city = re.sub('</li>',' ~ ',str(city))
        #city = " ".join(city)
        city = cleanhtml(city)
    
    except:
        city = ""
    
    # location
    try:
        location = re.findall('<p>Location</p>\s*<h6>(.*?)</h6>.*?</div>',str(spage))[0]
        location = re.sub('</li>\s*</ul>','',str(location))
        location = re.sub('</li>',' ~ ',str(location))
        #location = " ".join(location)
        location = cleanhtml(location)
    
    except:
        location = ""
    
    # pincode
    try:
        pincodes = re.findall('\d{5,6}',str(location))
        pincode = pincodes[0]
    except:
        pincode = ""
    
    # state
    try:
        states = re.findall(',\s*([A-Za-z\s]+)\.$',str(location))
        state = states[0]
    except:
        state = ""
    
    # location substitution
    try:
        location = re.sub('\-\d{6}\,','',str(location)) # common pincode removal
        location = re.sub(str(state),'',str(location)) #common state removal
        location = re.sub('\,\s*\d{6}\s,([A-Za-z\s]+)\-\s\d{6}\,\.','.',str(location)) # belagavi
        location = re.sub('\,([A-Za-z\s]+)\s\d{6}','',str(location)) # deesa gujrat
        location = re.sub('\s\,([A-Za-z\s]+)\s\.','.',str(location)) #deesa gujrat
        location = re.sub('\,([A-Za-z\s]+)\-\d{6} ,','',str(location)) # faridabad
        location = re.sub('\s\d{6}\,([A-Za-z\s]+)','',str(location)) # moholi
        location = re.sub('\s\,\-\d{5}\,','',str(location))# nepal
        location = re.sub('\,\s','',str(location)) # nepal
        location = re.sub('(\,Patna\.)$','.',str(location))# patna
        location = re.sub('\,\,',', ',str(location)) # panipat
        location = re.sub('\.\,',', ',str(location)) # telangana shine up
        location = re.sub('\,\.','.',str(location)) # hyderabad
        location = re.sub('vadodaravadodara','vadodara',str(location)) # vadodara
        
    except:
        location = ""
    
    # contact person
    try:
        contact_person = re.findall('<p>Contact Person</p>\s*<h6>(.*?)</h6>.*?</div>',str(spage))[0]
        contact_person = re.sub('</li>\s*</ul>','',str(contact_person))
        #contact_person = re.sub('</li>',' ~ ',str(contact_person))
        #contact_person = " ".join(contact_person)
        contact_person = cleanhtml(contact_person)
        contact_persons = contact_person.split(',')
                
    except:
        contact_person = ""
    
    # phone 
    try:
        phone = re.findall('<p>Phone Number</p>\s*<h6>(.*?)</h6>.*?</div>',str(spage))[0]
        phone = re.sub('</li>\s*</ul>','',str(phone))
        #phone = re.sub('</li>',' ~ ',str(phone))
        #phone = " ".join(phone)
        phone = cleanhtml(phone)
        phones = phone.split(',')
        
    except:
        phone = ""
    
    # email
    try:
        email = re.findall('<p>Email Address</p>\s*<h6>(.*?)</h6>.*?</div>',str(spage))[0]
        email = re.sub('</li>\s*</ul>','',str(email))
        email = re.sub('</li>',' ~ ',str(email))
        #email = " ".join(email)
        email = cleanhtml(email)
                
    except:
        email = ""
    
    # timing 1
    try:
        timing1 = re.findall('<p>Opening and Closing Time</p>\s*<h6>(.*?)</h6>.*?</div>',str(spage))[0]
        timing1 = re.sub('</li>\s*</ul>','',str(timing1))
        timing1 = re.sub('</li>',' ~ ',str(timing1))
        #timing1 = " ".join(timing1)
        timing1 = cleanhtml(timing1)
                
    except:
        timing1 = ""
    
    # timing 2
    try:
        timing2 = re.findall('<p>Opening and Closing Time</p>\s*<h6>(.*?)</h6>.*?</div>',str(spage))[1]
        timing2 = re.sub('</li>\s*</ul>','',str(timing2))
        timing2 = re.sub('</li>',' ~ ',str(timing2))
        #timing2 = " ".join(timing2)
        timing2 = cleanhtml(timing2)
                
    except:
        timing2 = ""
    
    # latitude
    try:
        latitude = re.findall('"latitude":"(.*?)"',str(spage))[0]
        latitude = re.sub('</li>\s*</ul>','',str(latitude))
        latitude = re.sub('</li>',' ~ ',str(latitude))
        #latitude = " ".join(latitude)
        latitude = cleanhtml(latitude)
                
    except:
        latitude = ""
    
    # longitude
    try:
        longitude = re.findall('"longitude":"(.*?)"',str(spage))[0]
        longitude = re.sub('</li>\s*</ul>','',str(longitude))
        longitude = re.sub('</li>',' ~ ',str(longitude))
        #longitude = " ".join(longitude)
        longitude = cleanhtml(longitude)
                
    except:
        longitude = ""
    
    my_dict = {}
    my_dict['Company_Name'] = company_name
    my_dict['City'] = city.strip()
    my_dict['Address'] = location
    my_dict['State'] = state
    my_dict['Pincode'] = pincode
    my_dict['Contact_Person_1'] = contact_persons[0]
    my_dict['Contact_Person_2'] = contact_persons[1]
    my_dict['Mobile1'] = phones[0].strip()
    my_dict['Mobile2'] = phones[1].strip()
    my_dict['Email'] = email
    my_dict['Opening_Closing_Time'] = timing1
    my_dict['Weekly_Services'] = timing2
    my_dict['Company_Url'] = main_cat
    
    df = pd.DataFrame(my_dict, index=[0])
    list1=list(my_dict.values())
    print(f"writing to csv .... ")
    #count+=1
    with open(out_csv, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header_list,)
        if file.tell() == 0:
            writer.writeheader()# Write the data row
        writer.writerow(my_dict) 
        #df['Mobile1'] = df['Mobile1'].apply(lambda x: x.replace(" ", "").strip())

In [462]:
file_exists = exists('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html')
if file_exists:
        pass
else:
    import requests
    homepage = requests.get(baseurl)
    print(homepage.status_code)
    if homepage.status_code == 200:
            with open('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html','w')as f:
                
                f.write(homepage.text.encode(encoding="utf-8",errors='ignore').decode())
                
HTMLFile = open('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html', "r")
hpage = HTMLFile.read()
hpage = re.sub('\\n','',str(hpage))
hpage = re.sub('<!--(.*?)-->','',str(hpage))
hpage = re.sub('#038;','',str(hpage))
mhash = re.findall('<div class="officeaddress one".*?<h4>(.*?)</h4>\s*<a href="(.*?)".*?</a>',str(hpage))
main_cat_hash = dict(mhash)
print(main_cat_hash)

{'"Ajmer"': 'https://www.exppresscarwash.com/locateOurOutlets/Ajmer', '"Belagavi"': 'https://www.exppresscarwash.com/locateOurOutlets/Belagavi', '"Bhubneshwar"': 'https://www.exppresscarwash.com/locateOurOutlets/Bhubneshwar', '"Bodhgaya"': 'https://www.exppresscarwash.com/locateOurOutlets/Bodhgaya', '"Darbhanga"': 'https://www.exppresscarwash.com/locateOurOutlets/Darbhanga', '"Deesa Gujarat"': 'https://www.exppresscarwash.com/locateOurOutlets/DeesaGujarat', '"Dimapur"': 'https://www.exppresscarwash.com/locateOurOutlets/Dimapur', '"Faridabad"': 'https://www.exppresscarwash.com/locateOurOutlets/Faridabad', '"G B Nagar"': 'https://www.exppresscarwash.com/locateOurOutlets/GBNagar', '"Garhshankar"': 'https://www.exppresscarwash.com/locateOurOutlets/Garhshankar', '"Giddarbaha"': 'https://www.exppresscarwash.com/locateOurOutlets/Giddarbaha', '"Goa"': 'https://www.exppresscarwash.com/locateOurOutlets/Goa', '"Greater Noida West"': 'https://www.exppresscarwash.com/locateOurOutlets/GreaterNoidaWe

In [463]:
for main_cat_url in main_cat_hash:
    main_cat = main_cat_hash[main_cat_url]
    if re.match('http',str(main_cat)):
        pass
    else:
        main_cat = str(baseurl)+str(main_cat)
    print(main_cat)
    
    main_cat_url = main_cat
    main_cat_url = main_cat_url.replace('/','_').replace(':','_').replace('.','')
    print(main_cat_url)
    
    file_exists = exists('/home/justdial/data/input/'+str(input_folder)+'/'+str(main_cat_url)+'.html')
    if file_exists:
        pass
    else:
        main_cat_page = requests.get(main_cat)
        print(main_cat_page.status_code)
        if main_cat_page.status_code == 200:
            with open('/home/justdial/data/input/'+str(input_folder)+'/'+str(main_cat_url)+'.html','w')as f:
                f.write(main_cat_page.text.encode(encoding="utf-8",errors='ignore').decode())
    HTMLFile = open('/home/justdial/data/input/'+str(input_folder)+'/'+str(main_cat_url)+'.html', "r")
    spage = HTMLFile.read()
    spage = re.sub('\\n','',str(spage))
    fetch_details(spage,main_cat)

https://www.exppresscarwash.com/locateOurOutlets/Ajmer
https___wwwexppresscarwashcom_locateOurOutlets_Ajmer
writing to csv .... 
https://www.exppresscarwash.com/locateOurOutlets/Belagavi
https___wwwexppresscarwashcom_locateOurOutlets_Belagavi
writing to csv .... 
https://www.exppresscarwash.com/locateOurOutlets/Bhubneshwar
https___wwwexppresscarwashcom_locateOurOutlets_Bhubneshwar
writing to csv .... 
https://www.exppresscarwash.com/locateOurOutlets/Bodhgaya
https___wwwexppresscarwashcom_locateOurOutlets_Bodhgaya
writing to csv .... 
https://www.exppresscarwash.com/locateOurOutlets/Darbhanga
https___wwwexppresscarwashcom_locateOurOutlets_Darbhanga
writing to csv .... 
https://www.exppresscarwash.com/locateOurOutlets/DeesaGujarat
https___wwwexppresscarwashcom_locateOurOutlets_DeesaGujarat
writing to csv .... 
https://www.exppresscarwash.com/locateOurOutlets/Dimapur
https___wwwexppresscarwashcom_locateOurOutlets_Dimapur
writing to csv .... 
https://www.exppresscarwash.com/locateOurOutlet

In [464]:
cut1 = re.findall('<div class="col-md-6 bg-yellow">(.*?)<div class="col-md-2">',str(spage))[0]
print(cut1)

                <div class="contact-text">                    <h2> <span>Exppress car Wash <span class="address-text"> vadodara</span></span> </h2>                    <br>                    <div class="col-md-12">                        <div class="contact-info">                            <div class="contact-item address">                                <div class="row">                                    


NameError: name 'phone' is not defined

In [80]:
print(company_name)

Exppress car Wash 
